# Objetivos deste trabalho
- Familiarizar-se com a biblioteca PyTorch
- Definir arquiteturas MLP simples em PyTorch
- Treinar utilizando CIFAR10, testando diferentes arquiteturas, parâmetros, funções de loss e otimizadores
- Comparar os resultados obtidos utilizando apenas Perpceptrons

In [1]:
%matplotlib inline

import numpy as np 
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [2]:
# Carregar os datasets

transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

dataset_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

dataset_test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
train_loader = DataLoader(dataset=dataset_train, shuffle=True, batch_size=512)
test_loader = DataLoader(dataset=dataset_test, shuffle=False)

In [4]:
# Definir a arquitetura MLP

class MLP(nn.Module):

    def __init__(self, layers):
        super(MLP, self).__init__()
        self.layers = layers
        self.fc1 = nn.Linear(32*32, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 10)
        self.activation_function = torch.nn.ReLU()
        
    def forward(self, x):
        x = x.view(-1, 32*32)
        x = self.activation_function(self.fc1(x))
        x = self.activation_function(self.fc2(x))
        x = self.activation_function(self.fc3(x))
        x = self.activation_function(self.fc4(x))
        return x
    
def one_hot(label, shape):
    o_hot = np.zeros(shape)
    o_hot[label] = 1
    return o_hot
    

In [5]:
model = MLP([])
print(model)

MLP(
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=10, bias=True)
  (activation_function): ReLU()
)


In [6]:
# Definir otimizador e loss
# Nota: testar outros otimizadores e funções de loss (em particular cross entropy)

optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)
loss_fn = torch.nn.CrossEntropyLoss()

In [9]:
if torch.cuda.is_available():
    model = model.cuda()

# Realizar o treinamento aqui
for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        print(labels)
        if torch.cuda.is_available():
            inputs = torch.cuda.FloatTensor(np.array(inputs))
            labels = torch.cuda.FloatTensor(np.array(labels))
        else:
            inputs = torch.FloatTensor(np.array(inputs))
            labels = torch.FloatTensor(np.array(labels))
        
        optimizer.zero_grad()
        y_pred = model(inputs)
        loss = loss_fn(y_pred, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
    print(epoch + 1, running_loss / 50000) 
    

tensor([0, 3, 6, 5, 4, 4, 3, 1, 6, 5, 6, 0, 9, 1, 8, 8, 4, 6, 2, 0, 6, 9, 8, 4,
        3, 9, 8, 4, 3, 0, 8, 7, 3, 0, 0, 8, 5, 7, 3, 1, 5, 3, 3, 7, 9, 9, 0, 5,
        4, 1, 8, 0, 5, 2, 5, 1, 4, 6, 1, 9, 0, 4, 3, 6, 4, 6, 1, 8, 8, 0, 5, 6,
        7, 9, 1, 5, 1, 1, 0, 9, 7, 3, 2, 6, 7, 8, 0, 9, 7, 1, 2, 6, 6, 3, 3, 6,
        1, 8, 8, 9, 9, 9, 4, 6, 4, 4, 7, 5, 5, 0, 0, 8, 2, 1, 7, 8, 3, 8, 6, 0,
        7, 7, 2, 1, 4, 1, 7, 1, 2, 3, 3, 7, 7, 3, 0, 5, 4, 6, 0, 9, 0, 2, 2, 5,
        6, 3, 6, 6, 5, 4, 1, 4, 3, 7, 3, 0, 0, 8, 6, 7, 0, 0, 0, 4, 3, 2, 2, 3,
        0, 1, 8, 2, 6, 5, 4, 0, 3, 3, 3, 5, 9, 9, 8, 7, 2, 4, 7, 7, 0, 2, 2, 5,
        7, 6, 2, 2, 5, 0, 3, 5, 2, 7, 3, 0, 9, 0, 9, 7, 2, 5, 9, 4, 5, 1, 6, 4,
        9, 1, 5, 8, 7, 4, 4, 5, 9, 7, 7, 4, 2, 9, 5, 1, 1, 5, 7, 1, 3, 6, 2, 4,
        2, 0, 1, 9, 3, 3, 9, 8, 3, 0, 3, 5, 3, 0, 0, 6, 3, 0, 9, 6, 3, 6, 8, 2,
        6, 3, 6, 7, 0, 4, 4, 2, 3, 7, 2, 3, 0, 5, 6, 6, 1, 0, 8, 7, 0, 5, 4, 0,
        5, 3, 2, 4, 2, 1, 5, 4, 5, 2, 1,

RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'target'

In [ ]:
# Avaliar o modelo aqui (no conjunto de teste)
for i, data in enumerate(test_loader):
    inputs, labels = data
    labels = one_hot(labels, 10)
    if torch.cuda.is_available():
        inputs = torch.cuda.FloatTensor(np.array(inputs))
        labels = torch.cuda.FloatTensor(np.array(labels))
    else:
        inputs = torch.FloatTensor(np.array(inputs))
        labels = torch.FloatTensor(np.array(labels))

    pred = model(inputs)
    
    results = list(map(lambda x: x.item(), pred[0]))
    result = (results.index(max(results)))
    print(result)